In [2]:
import porter 
from collections import Counter
import math

### Exercice 1 – Indexation d’un petit jeu de données


On considère les documents pr´esent´es lors du TD1 :

— Doc 1 : the new home has been saled on top forecasts

— Doc 2 : the home sales rise in july

— Doc 3 : there is an increase in home sales in july

— Doc 4 : july encounter a new home sales rise

Ainsi qu’une liste de mots vides : the, a, an, on, behind, under, there, in, on.

Q 1.1 Ecrire le code qui a partir de la chaˆıne de caractere du document 1 : 
1) s´epare les mots, les transforme en minuscule, compte le nombre d’occurrences par mot dans le texte, 
2) supprime les mots vides et 
3) stocke le resultat sous la forme :

1 {’new ’ : 1 , ’home ’ : 1 , ’ha’ : 1 , ’been ’ : 1 , ’sale ’ : 1 , ’top ’ : 1 , ’forecast ’ : 1}

Remarque : pour la normalisation des termes, on s’aidera du fichier porter.py. Pour compter le nombre d’occurrences d’un terme, on utilisera la librairie Counter de collection.


In [3]:
Corpus = [ "the new home has been saled on top forecasts",
              "the home sales rise in july",
              "there is an increase in home sales in july",
              "july encounter a new home sales rise"]

def preprocessing(doc) : 
    words = doc.split()
    words = [porter.stem(word) for word in words]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in ["the","a","an","on","in","has","been","there","is","a","an"]]
    
    return Counter(words)

# exemple
preprocessing(Corpus[0])

Counter({'new': 1, 'home': 1, 'ha': 1, 'sale': 1, 'top': 1, 'forecast': 1})

In [4]:
# indexation

# index 
def create_index(doc) : 
    index = {}
    for i,doc in enumerate(Corpus) : 
        index[i] = preprocessing(doc)
    return index

# inverted index
def create_inverted_index(index) : 
    inverted_index = {}
    for i,doc in index.items() : 
        for word in doc : 
            if word not in inverted_index : 
                inverted_index[word] = {}
            if inverted_index[word].get(i) is None:
                inverted_index[word][i] = 1
            else :
                inverted_index[word][i] += 1
    
    return inverted_index
        
# affichage index et inverted index
print(create_index(Corpus))
print(create_inverted_index(create_index(Corpus)))

{0: Counter({'new': 1, 'home': 1, 'ha': 1, 'sale': 1, 'top': 1, 'forecast': 1}), 1: Counter({'home': 1, 'sale': 1, 'rise': 1, 'juli': 1}), 2: Counter({'increas': 1, 'home': 1, 'sale': 1, 'juli': 1}), 3: Counter({'juli': 1, 'encount': 1, 'new': 1, 'home': 1, 'sale': 1, 'rise': 1})}
{'new': {0: 1, 3: 1}, 'home': {0: 1, 1: 1, 2: 1, 3: 1}, 'ha': {0: 1}, 'sale': {0: 1, 1: 1, 2: 1, 3: 1}, 'top': {0: 1}, 'forecast': {0: 1}, 'rise': {1: 1, 3: 1}, 'juli': {1: 1, 2: 1, 3: 1}, 'increas': {2: 1}, 'encount': {3: 1}}


In [5]:
def create_indexTfIdf(corpus, index, inverted_index) : 
    N = len(corpus)
    indexTfIdf = {}
    for i,doc in index.items() : 
        indexTfIdf[i] = {}
        for word in doc : 
            indexTfIdf[i][word] = (doc[word]/len(doc)) * ( math.log(N/len(inverted_index[word])))
    return indexTfIdf

# exemple
create_indexTfIdf(Corpus, create_index(Corpus), create_inverted_index(create_index(Corpus)))

{0: {'new': 0.11552453009332421,
  'home': 0.0,
  'ha': 0.23104906018664842,
  'sale': 0.0,
  'top': 0.23104906018664842,
  'forecast': 0.23104906018664842},
 1: {'home': 0.0,
  'sale': 0.0,
  'rise': 0.17328679513998632,
  'juli': 0.07192051811294521},
 2: {'increas': 0.34657359027997264,
  'home': 0.0,
  'sale': 0.0,
  'juli': 0.07192051811294521},
 3: {'juli': 0.04794701207529681,
  'encount': 0.23104906018664842,
  'new': 0.11552453009332421,
  'home': 0.0,
  'sale': 0.0,
  'rise': 0.11552453009332421}}

### Exercice 2 - Modèles de RI

On considère la collection de documents et la liste des stopwords de l’exercice precedent. L’objectif dans cet exercice est d’estimer le score des documents pour la requete ”home sales top”.

Q 2.1 On pensera a l’optimisation du calcul du score. Quels index faut-il interroger pour avoir un calcul du score
pertinent ?

<tt> **Il faudrait interroger l'indexe inversé pour avoir un calcul du score pertinent.** <tt>

Q 2.2 Ecrire le code qui permet de calculer le score des documents a partir du modele booleen.

• Modèle pionnier

• Basé sur la théorie des ensembles

• Représentation logique des documents L(d) et des requêtes L(q) en

utilisant les opérateurs logiques : OU (∨), ET (∧) et NON (¬).

• Exemple :


– d1(t1,t3,t5) ; d2(t1,t3,t5) ; d3(t1,t2,t3,t4)

• Score de similarité :

RSV(q, d) = 
1 si L(q) ⊂ L(d)
0 sinon


In [6]:
index = create_index(Corpus)
inverted_index = create_inverted_index(create_index(Corpus))
indexTfIdf = create_indexTfIdf(Corpus, index, inverted_index)

In [7]:

def bool_doc_scores(q, inverted_index) : 
    docs = set ()
    for word in q : 
        if word in inverted_index.keys() and not docs: 
            docs.update(inverted_index[word].keys())
        elif word in inverted_index.keys() :
            docs = docs.intersection(inverted_index[word].keys())
        else :
            return "no documents"
    return docs
            
    
# exemple
q = "home sales rise july"
q = preprocessing(q) 
bool_doc_scores(q, inverted_index)

{1, 3}

Q 2.3 Ecrire le code qui permet de calculer le score des documents a partir du modele vectoriel (produit scalaire)
dans le cas d’une pond´eration tf.

In [8]:
def vect_doc_scores(q, indexTfIdf, index, inverted_index) : 
    scores = {}
    docs = set()
    for word in q : 
        if word not in inverted_index.keys() : 
            continue
        docs.update(inverted_index[word].keys())

    for doc in docs :
        scores[doc] = 0
        for word in q : 
            if word in index[doc].keys() : 
                scores[doc] += indexTfIdf[doc][word]
    return scores
    
# exemple
q = ['home','sale', 'top']

vect_doc_scores(q, indexTfIdf, index, inverted_index)

{0: 0.23104906018664842, 1: 0.0, 2: 0.0, 3: 0.0}

Q 2.4 Ecrire le code qui permet de calculer le score des documents `a partir du modele de langue Jelineck-Mercer.

$$ score(Q,D) = \prod_{i=1}^{n} [(1 - \lambda) P(t_i|D) + \lambda P(t_i|C)] $$
$$ P(t_i|D) = \frac{tf_{t_i,D}}{|D|} $$
$$ P(t_i|C) = \frac{tf_{t_i,C}}{|C|} $$
$ \lambda $ est un paramètre de lissage, $ P(t_i|D) $ est la probabilité d’apparition du terme $ t_i $ dans le document D, $ P(t_i|C) $ est la probabilité d’apparition du terme $ t_i $ dans la collection C.

In [9]:
def jelineck_mercer_score(query, index, inverted_index, corpus, lambda_param=0.5) : 
    score = {}
    docs = set()
    taille_corpus = sum(len(index[i]) for i, _ in enumerate(corpus))

    for word in query : 
        if word not in inverted_index.keys() : 
            continue
        docs.update(inverted_index[word].keys())

    for doc in docs :
        score[doc] = 1.0
        taille_doc = len(index[doc])
        
        for word in query : 
            if word not in index[doc].keys() :
                continue

            p_w_d = index[doc].get(word) / taille_doc
            p_w_c = len(inverted_index[word]) / taille_corpus

            score[doc] *= lambda_param * p_w_d + (1 - lambda_param) * p_w_c
    return score

# exemple
q = "home sales rise july"
q = preprocessing(q) 
jelineck_mercer_score(q, index, inverted_index, Corpus)

{0: 0.03361111111111112,
 1: 0.001771875,
 2: 0.010125000000000002,
 3: 0.000709567901234568}

### Exercice 3 – Evaluation en RI
L’objectif dans cet exercice est de mesurer la qualité de l’ordonnancement. Pour cela, on définit les jugements de
pertinence suivant :

— Requete 1 ”top sales” - Documents pertinents : 1 <br>
— Requete 2 ”sales increase july” - Documents pertinents : 2 et 3 (avec 2 plus pertinent que 3) <br>
— Requete 3 ”new home”

Q 3.1 Calculer les mesures de precision, rappel et F-mesure au rang 2 (P@2, R@2 et F@2) pour chaque requete et ensuite leur moyenne sur l’ensemble des reqetes.
* Précision au rang 2 (P@2) :
$$ P@2 = \frac{\text{Nombre de documents pertinents retrouvés dans les 2 premiers rangs}}{\text{2}} $$
* Rappel au rang 2 (R@2) :
$$ R@2 = \frac{\text{Nombre de documents pertinents retrouvés dans les 2 premiers rangs}}{\text{Nombre total de documents pertinents}} $$
* F-mesure au rang 2 (F@2) :
$$ F@2 = \frac{2 \times P@2 \times R@2}{P@2 + R@2} $$



In [10]:
def metric(q, doc_pertinent, rang) :
    scores =  jelineck_mercer_score(preprocessing(q), index, inverted_index, Corpus)
    ranking = sorted(scores, key=scores.get, reverse=True)
    TP = 0
    
    rang = min(rang, len(ranking))
    for i in range(rang) : 
        if ranking[i] in doc_pertinent :
            TP += 1

    precision = TP / rang
    rappel = TP / len(doc_pertinent)

    return precision, rappel

precision, rappel = metric("top sales", doc_pertinent=[0], rang=2)
print("precision : ", precision, "rappel : ", rappel)

precision :  0.0 rappel :  0.0


Q 3.2 Calculer la mesure de NDCG pour toutes les requetes

$$
nDCG_{k} = \frac{DCG_{k}}{IDCG_{k}}
$$

$$
DCG_{k} = rel_1 + \sum_{i=2}^{k} \frac{rel_i}{\log_2(i)}
$$




In [ ]:
def IDCG(q, doc_pertinent, rang) : 
    scores =  jelineck_mercer_score(preprocessing(q), index, inverted_index, Corpus)
    ranking = sorted(scores, key=scores.get, reverse=True)
    IDCG = 0
    rang = min(rang, len(ranking))
    for i in range(rang) : 
        if ranking[i] in doc_pertinent :
            IDCG += 1 / math.log2(i+2)
    return IDCG

In [11]:

# exemple
IDCG = 2 + 1/math.log(2)
query1 = "top sales"
query2 = "query increase july"
query1 = "new home"
nDCG(query2, doc_pertinent={1:2 , 2:1}, rang=2, IDCG=IDCG)

0.5809402158035948